In [645]:
import requests
import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', None)


league_id = 2101568920
my_swid = "FE436ABA-706C-409F-BFD8-69726D7E3F52"
my_espn_s2 = "AEA36ktSbqPyLKlmB5C6RIKP0DEpnrgEQTIJe4qAlFLpl6IeOAk6SS6T2n1XAiQ3XhyBicLFQOninj74dwRaOhULVy8eAOOWAbn2HVTjoEA%2F2E4crW05i30ZunDbMysj4VzigcJAF%2Bh%2BSwtawTL40AbjpJkWaYnMYC92cEW%2FgD6h32O8Rf5tdemmFq7TCQMsDRiMSSND3KKTUrU4lichHqrTxJxCi9h92T47ihJ6NTL7irf8Oa4zxTjzdFuSUn8ZoUf0Fi%2FBPYbTjg6fs1o34LVY"
year = 2023

url = "https://fantasy.espn.com/apis/v3/games/FBA/seasons/" + \
      str(year) + "/segments/0/leagues/" + str(league_id) 

Some of available views: mTeam,
mMatchup,
mRoster,
mSettings,
mBoxscore,
mMatchupScore,
kona_player_info,
player_wl,
mSchedule,
mScoreboard

In [646]:

filters = {'players': { 'limit': 1500, 'sortDraftRanks': { "sortPriority": 100, "sortAsc": True, "value": "STANDARD" }}}
params={"view": ["kona_player_info"]}
player_info = requests.get(url,
                 params=params,
                 headers={'x-fantasy-filter': json.dumps(filters)},
                 cookies={"swid":my_swid ,
                          "espn_s2":my_espn_s2})
player_info = player_info.json()

In [647]:
df = pd.DataFrame.from_dict(player_info['players'])
df=df[['id','onTeamId','player','ratings','status','waiverProcessDate']]
player_df = pd.json_normalize(df['player'])
player_df = player_df[['defaultPositionId', 'eligibleSlots',
        'fullName', 'injured', 'injuryStatus', 'proTeamId',
       'stats', 'ownership.percentOwned']]

In [650]:
def process_2023_stats(l):
    return l[0]
def process_7day_stats(l):
    return l[2]
def process_15day_stats(l):
    return l[1]
# def process_30day_stats(l):
#     return l[4]

player_df['2023 stats'] = player_df['stats'].apply(process_2023_stats)
player_df['7 day stats'] = player_df['stats'].apply(process_7day_stats)
player_df['15 day stats'] = player_df['stats'].apply(process_15day_stats)
# player_df['30 day stats'] = player_df['stats'].apply(process_30day_stats)

stats_2023_df = pd.json_normalize(player_df['2023 stats'])
stats_2023_df = stats_2023_df[['appliedAverage','appliedTotal','averageStats.40']]

In [651]:
player_df = pd.concat([player_df,stats_2023_df],axis=1)
player_df = pd.concat([df, player_df],axis=1)

In [652]:
player_df = player_df[['fullName','injuryStatus','status','defaultPositionId','proTeamId','averageStats.40', 'appliedAverage', 'appliedTotal', 'ownership.percentOwned']]

In [653]:
player_df = player_df.rename(columns={'averageStats.40':'avg. minutes','appliedAverage':'avg. ftps','appliedTotal':'total fpts','ownership.percentOwned':'rostered'})

In [654]:
player_df.loc[player_df['defaultPositionId'] ==2,'defaultPositionId'] = 'SG'
player_df.loc[player_df['defaultPositionId'] ==1,'defaultPositionId'] = 'PG'
player_df.loc[player_df['defaultPositionId'] ==3,'defaultPositionId'] = 'SF'
player_df.loc[player_df['defaultPositionId'] ==4,'defaultPositionId'] = 'PF'
player_df.loc[player_df['defaultPositionId'] ==5,'defaultPositionId'] = 'C'

## INJURY REPLACEMENTS

In [659]:
relevant_players = player_df[player_df['total fpts'] != 0.0]
relevant_players = relevant_players[relevant_players['avg. minutes']>5]
relevant_players = relevant_players[relevant_players['avg. ftps']>5]
relevant_players.reset_index()
print()

In [660]:
injured_players = relevant_players.loc[(relevant_players['injuryStatus']=='OUT')&(relevant_players['avg. minutes']>25)]

In [661]:
injured_players

,fullName,injuryStatus,status,defaultPositionId,proTeamId,avg. minutes,avg. ftps,total fpts,rostered
12,Devin Booker,OUT,ONTEAM,SG,21,35.607143,34.464286,965.0,99.947898
24,Anthony Davis,OUT,ONTEAM,PF,13,33.440000,46.440000,1161.0,99.839655
33,Stephen Curry,OUT,ONTEAM,PG,9,34.423077,43.230769,1124.0,99.928108
34,Karl-Anthony Towns,OUT,ONTEAM,C,16,33.761905,33.380952,701.0,99.716064
35,Andrew Wiggins,OUT,ONTEAM,SF,9,32.772727,29.045455,639.0,95.101264
48,Tyrese Maxey,OUT,ONTEAM,SG,20,36.333333,27.266667,409.0,88.239761
55,Brandon Ingram,OUT,ONTEAM,SF,3,32.066667,27.733333,416.0,94.680744
99,Kelly Olynyk,OUT,ONTEAM,C,26,28.531250,23.406250,749.0,40.003798
110,Jalen Suggs,OUT,ONTEAM,PG,19,28.928571,21.642857,303.0,51.616538
137,Dennis Smith Jr.,OUT,FREEAGENT,PG,30,28.000000,24.333333,365.0,18.498315


In [662]:
for index, row in injured_players.iterrows():
    print('INJURED_PLAYER:', row['fullName'], ', Primary Position:', row['defaultPositionId'])
    print(" ")
    team_id = row['proTeamId']
    team_df = relevant_players[relevant_players['proTeamId']==team_id]
    for index_2, row_2 in team_df.iterrows():
        if row_2['status']=='FREEAGENT' and row_2['injuryStatus']=='ACTIVE':
            print(row_2['fullName'],' ',round(row_2['avg. ftps'],2),' ',round(row_2['avg. minutes'],2),' ',row_2['defaultPositionId'])
        if row_2['status']=='WAIVERS':
            print(row_2['fullName'],' ',round(row_2['avg. ftps'],2),' ',round(row_2['avg. minutes'],2),' ',row_2['defaultPositionId'],' (ON WAIVERS)')


    print('----------------')

INJURED_PLAYER: Devin Booker , Primary Position: SG
 
Damion Lee   13.03   21.45   SF
Bismack Biyombo   11.74   13.78   C
Dario Saric   7.25   11.06   PF
Ish Wainright   9.19   13.12   PF
Josh Okogie   10.14   11.11   SG
Jock Landale   11.93   13.43   C
----------------
INJURED_PLAYER: Anthony Davis , Primary Position: PF
 
Damian Jones   7.76   9.65   C
Troy Brown Jr.   10.66   22.59   SF
Max Christie   6.56   14.31   SG
Wenyen Gabriel   12.0   14.26   SF
----------------
INJURED_PLAYER: Stephen Curry , Primary Position: PG
 
Donte DiVincenzo   15.52   20.61   SG
Jonathan Kuminga   10.72   18.21   PF
James Wiseman   11.12   13.5   C
Moses Moody   8.0   16.0   SG
Anthony Lamb   11.64   18.28   SF
Ty Jerome   10.65   16.18   PG
Patrick Baldwin Jr.   5.44   7.44   SF
----------------
INJURED_PLAYER: Karl-Anthony Towns , Primary Position: C
 
Naz Reid   16.19   16.78   C
Austin Rivers   8.74   18.57   SG  (ON WAIVERS)
----------------
INJURED_PLAYER: Andrew Wiggins , Primary Position: SF
